In [53]:
import pandas as pd
from collections import Counter as c
import numpy as np

In [54]:
df = pd.read_csv('ab_pred_asp_cats.csv') # файл с сентиментом для эксплицитных аспектов
gold = pd.read_csv('ab_dev_cats.txt', sep='\t', header=None) # таргет

In [77]:
def switch(labs): # приоритеты при равных скорах
    if 'negative' in labs and 'positive' in labs:
        return 'both'
    elif 'positive' in labs:
        return 'positive'
    elif 'negative' in labs:
        return 'negative'
    elif 'both' in labs:
        return 'both'
    elif 'neutral' in labs:
        return 'neutral'
    

In [79]:
def cats_from_asp(df, gold):
    vals = df.values
    d = {}

    for i in vals:
        if not i[0] in d:
            d[i[0]] = [(i[1], i[5])]
        else:
            d[i[0]].append((i[1], i[5]))


    for i in d:
        d[i] = dict(c(d[i]))
        
        
    res = []
    cats = gold[1].unique().tolist()
    for i in d:
        id_ = i
        for j in cats:
            grades = [k for k in d[i] if k[0]==j]

            if not grades:
                ln = [id_, j, 'absence']
            else:
                scores = np.asarray([d[i][k] for k in grades])  # приписывается самая частая метка категории     
                idx = np.argmax(scores)
                
                labs = [k[1] for k in grades if d[i][k]==scores[idx]]
                if len(labs)==1:
                    sent = grades[idx][1]
                    ln = [id_, j, sent]
                else:
                    fl = switch(labs)
                    ln = [id_, j, fl]

            res.append(ln)
            
    out_df = pd.DataFrame(res, columns = ['id', 'cat', 'sent'])
    return out_df


In [81]:
from sklearn.metrics import accuracy_score
out_df = cats_from_asp(df, gold)
a = accuracy_score(gold[2].tolist(), out_df['sent'].tolist())
print(f'acc: {a}')

acc: 0.7830985915492957
